In [15]:
from selenium import webdriver
import time

#파일 경로
save_path = 'c:/cr/app/'
driver_path = 'c:/cd/chromedriver'

# 파일형식
save_type = 'csv'

#헤더 설정
user_agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) " +\
"AppleWebKit/537.36 (KHTML, like Gecko) " + \
"Chrome/37.0.2062.94 Safari/537.36"
headers = {"User-Agent": user_agent}

#접속 URL
#게임: 최고매출
url = 'https://play.google.com/store/apps/category/GAME/collection/topgrossing'
#찾고싶은 앱의 순위
index = 2
#찾고싶은 페이지 수 (기본 120개, 반복 1회당 120개 추가) ex. 1일때 240개, 2일때 360개의 리뷰를 가져옴
repeat = 2

In [2]:
browser = webdriver.Chrome(driver_path)
browser.implicitly_wait(3)
browser.get(url)
#body-content > div.outer-container > div > div.main-content > div > div > div > div.id-card-list.card-list.two-cards > div:nth-child(1)
#body-content > div.outer-container > div > div.main-content > div > div > div > div.id-card-list.card-list.two-cards > div:nth-child(2)

In [12]:
class ARcrawler:
    def __init__(self, url, index):
        self.url = url
        self.index = index
        
    
    def click_app(browser, index=1, order=1):
        # index 순위의 앱 클릭
        browser.find_element_by_xpath('//*[@id="body-content"]/div/div/div[1]/div/div/div/div[2]/div[{}]'.format(index)).click()
        browser.implicitly_wait(3)
        #리뷰 모두보기 클릭
        browser.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz/div/div[2]/div/div[1]/div/div/div/div[6]/div/content/span').click()
        #리뷰를 원하는 방식으로 정렬. 기본은 최신. (1. 최신 2. 높은 평점순 3. 유용도순)
        browser.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div[1]/div/div/div/div[2]/c-wiz/div/div/div[1]/div[2]/span').click()
        browser.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div[1]/div/div/div/div[2]/c-wiz/div/div/div[2]/div[{}]'.format(order)).click()
        return(index)
    
    
    def page_scroll(browser):
        SCROLL_PAUSE_TIME = 1

        # Get scroll height
        last_height = browser.execute_script("return document.body.scrollHeight")

        while True:
            # Scroll down to bottom
            browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = browser.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
    
    
    def more_pages(browser, repeat=1):
        for i in range(repeat):
            ARcrawler.page_scroll(browser)
            browser.implicitly_wait(5)
            browser.find_element_by_css_selector('div.JNury.Ekdcne > div > div > div > div > div.Ir7OJc > div > content > span').click()
            
            
    def get_reviews(browser):
        results = []
        ids = browser.find_elements_by_css_selector('div.JNury.Ekdcne > div > div > div > div > div > div > div > div.QV3svb.GVFJbb > div.H9roif.Boieuf > div.GVFJbb.jBjpkf > span')
        rates = browser.find_elements_by_css_selector('div.JNury.Ekdcne > div > div > div > div > div > div > div > div.QV3svb.GVFJbb > div.H9roif.Boieuf > div.GVFJbb.jBjpkf > div > span.qC3s2c > div > div')
        dates = browser.find_elements_by_css_selector('div.JNury.Ekdcne > div > div > div > div > div > div > div > div.QV3svb.GVFJbb > div.H9roif.Boieuf > div.GVFJbb.jBjpkf > div > span.oldIDd')
        reviews = browser.find_elements_by_css_selector('div.JNury.Ekdcne > div > div > div > div > div > div > div > div.QV3svb.GVFJbb > div.Z8UXhc > span')
        
        for i in range(len(ids)):
            results.append([ids[i].text, rates[i].get_attribute('aria-label'), dates[i].text, reviews[i*2].text])
        
        return(results)
 

    def save(index):
        txt_path = save_path + '{}_app_reviews.txt'.format(index)
        with open(txt_path, 'w+', encoding = 'utf-8')as f:
            for result in results:
                f.write(','.join(result))
                f.write('\n')

In [16]:
ARcrawler.click_app(browser, index)
ARcrawler.more_pages(browser, repeat)
results = ARcrawler.get_reviews(browser)
ARcrawler.save(index)

In [17]:
len(results)

360